# Task-First Prompt Layout

When building AI agents and designing prompts, one simple principle dramatically improves model focus and response quality: **put the task first, then add context.**

Traditional prompts often bury the actual question under layers of background information, forcing models to process extensive details before understanding what they are being asked to do. This leads to responses that miss the core task or focus on peripheral details.

Task-first prompt layout inverts this structure: state the objective immediately, then provide supporting information. This ensures the model knows what it needs to accomplish before processing how to accomplish it.

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from typing import List, Optional
import os

### Initialize LLM for testing

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY", "").strip(), temperature=0)

### The problem: Details-first vs task-first
Let's see the difference empirically. We will create two versions of the same prompt - one that buries the task under context (traditional), and one that states the task first (better).

In [3]:
# DETAILS-FIRST (traditional - task buried at the end)
details_first = """
Context: You are analyzing financial data for a technology company operating in cloud 
services, enterprise software, and hardware segments.

Background: Q4 is typically the strongest quarter. R&D expenses increased 25% year-over-year.

Constraints:
- Only use fiscal year 2023 data
- Express values in millions USD
- Include year-over-year comparisons
- No forward-looking predictions

Metrics to consider: revenue breakdown, margins, R&D spending, customer acquisition costs

Data notes: All figures from audited statements. Some segment data was restated in Q1 2024.

Task: Identify the top 3 factors that most significantly impacted revenue growth in 2023.
"""

# TASK-FIRST (better - task stated immediately)
task_first = """
TASK: Identify the top 3 factors that most significantly impacted revenue growth in 2023.

Requirements:
- Only use fiscal year 2023 data  
- Express values in millions USD
- Include year-over-year comparisons

Context: Technology company with cloud, enterprise software, and hardware segments. 
Q4 typically strongest. R&D expenses increased 25% YoY. All data from audited statements.
"""

# Test both
print("1. DETAILS-FIRST PROMPT:")
print("-" * 80)
print(f"{details_first[:200]}...\n[Task appears after 400+ characters]")

response1 = llm.invoke([HumanMessage(content=details_first)])
print(f"\nResponse preview: {response1.content[:300]}...\n")

print("\n" + "="*80)
print("\n2. TASK-FIRST PROMPT:")
print("-" * 80)
print(f"{task_first[:150]}...\n[Task appears immediately]")

response2 = llm.invoke([HumanMessage(content=task_first)])
print(f"\nResponse preview: {response2.content[:300]}...")

1. DETAILS-FIRST PROMPT:
--------------------------------------------------------------------------------

Context: You are analyzing financial data for a technology company operating in cloud 
services, enterprise software, and hardware segments.

Background: Q4 is typically the strongest quarter. R&D ex...
[Task appears after 400+ characters]

Response preview: To analyze the financial data for the technology company and identify the top three factors that significantly impacted revenue growth in fiscal year 2023, we can consider the following metrics and insights:

### 1. **Revenue Breakdown by Segment**
   - **Cloud Services**: This segment likely experi...



2. TASK-FIRST PROMPT:
--------------------------------------------------------------------------------

TASK: Identify the top 3 factors that most significantly impacted revenue growth in 2023.

Requirements:
- Only use fiscal year 2023 data  
- Express...
[Task appears immediately]

Response preview: To identify the top 3 fa

Task-first prompts produce better task adherence. The model focuses on answering the question vs discussing context.

**What task-first prompt layout really is:**
- Put our task/question/instruction at the beginning of our prompt.
- Add supporting details (context, data, examples) after the task.

**The basic pattern:**
```
Task: [What you want the model to do]

Requirements: [Any constraints or specifications]

Context: [Background information needed to complete the task]
```

Models process information sequentially and knowing the objective first helps the model interpret context correctly. It is especially important for complex prompts with lots of context. Implementation in:
- RAG systems: User question → Retrieved documents.
- Chat systems: Task in system/user message → Context follows.

## Chat-based systems (system + user messages)
For chat models, the pattern follows the same principle: instructions/task in system message or early in user message, context follows.

In [4]:
# Method: Chat-based pattern
def chat_task_first(task: str, context: str) -> list:
    """
    Chat models: task/instructions in system message or start of user message.
    Context comes after or in separate message.
    """
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant. Follow user instructions carefully."
        },
        {
            "role": "user",
            "content": f"Task: {task}\n\nContext: {context}"
        }
    ]
    return messages

# Example
task = "Summarize the key findings from the quarterly report"
context = "Q3 revenue $42M, down 15% QoQ. Churn up to 12%. Competitor pricing pressure."

messages = chat_task_first(task, context)

print("Chat-Based Task-First Pattern:")
print("=" * 80)
for msg in messages:
    print(f"{msg['role'].upper()}: {msg['content']}\n")

Chat-Based Task-First Pattern:
SYSTEM: You are a helpful assistant. Follow user instructions carefully.

USER: Task: Summarize the key findings from the quarterly report

Context: Q3 revenue $42M, down 15% QoQ. Churn up to 12%. Competitor pricing pressure.



## How RAG systems use task-first prompts
In RAG systems, the pattern is the same: user question first (that is the task), then retrieved context second.

In [5]:
# Method: RAG pattern
def rag_prompt(user_question: str, retrieved_docs: List[str]) -> str:
    """
    Standard RAG pattern: question first, documents second.
    This is what production RAG systems actually look like.
    """
    # Task/question goes first
    prompt = f"Question: {user_question}\n\n"
    
    # Retrieved context goes second
    prompt += "Relevant information:\n"
    for i, doc in enumerate(retrieved_docs, 1):
        prompt += f"{i}. {doc}\n"
    
    prompt += "\nAnswer based on the provided information."
    
    return prompt

# Simulate RAG
user_question = "What caused the revenue decline in Q3?"
retrieved_docs = [
    "Q3 2023 revenue: $42M (down from $49.5M in Q2, a 15% decline)",
    "Customer churn increased from 5% to 8% in Q3 2023",
    "Competitor launched similar product at 30% lower price in July 2023"
]

rag_result = rag_prompt(user_question, retrieved_docs)
print("RAG Task-First Prompt:")
print("=" * 80)
print(rag_result)
print("\n" + "="*80)

RAG Task-First Prompt:
Question: What caused the revenue decline in Q3?

Relevant information:
1. Q3 2023 revenue: $42M (down from $49.5M in Q2, a 15% decline)
2. Customer churn increased from 5% to 8% in Q3 2023
3. Competitor launched similar product at 30% lower price in July 2023

Answer based on the provided information.

